In [1]:
import keras
import numpy as np
import scipy.io
from pathlib import Path
import os

Using TensorFlow backend.


## Open labels

In [2]:
path = 'C:\\Users\\Rodolfo\\Documents\\algoritmos-git\\custom_cnn\\Data\\Processados\\'

In [3]:
y_train = np.load(Path(path,'y_train.npy'))
y_test = np.load(Path(path, 'y_test.npy'))
y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)

In [4]:
from keras.utils import np_utils
# one-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [6]:
y_test

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

# Open data

In [7]:
path = Path(path, '100_100')
X_train_1 = np.load(Path(path, 'X_train_1.npy'))
X_train_2 = np.load(Path(path, 'X_train_2.npy'))
X_test_1 = np.load(Path(path, 'X_test_1.npy'))
X_test_2 = np.load(Path(path, 'X_test_2.npy'))
print(X_train_1.shape)
print(X_train_2.shape)
print(X_test_1.shape)
print(X_test_2.shape)

(2200, 100, 100)
(2200, 100, 100)
(1000, 100, 100)
(1000, 100, 100)


### Shape desejado: x_train shape: (45000, 32, 32, 3) -> n_obs, n_row, n_col, n_chan

In [8]:
n_obs, n_row, n_col = X_train_1.shape
n_chan=2
X_train = np.zeros((n_obs, n_row, n_col, n_chan))
X_train.shape

(2200, 100, 100, 2)

In [9]:
X_train[:,:,:,0]=X_train_1
X_train[:,:,:,1]=X_train_2
X_train.shape

(2200, 100, 100, 2)

In [10]:
n_obs, n_row, n_col = X_test_1.shape
n_chan=2
X_test = np.zeros((n_obs, n_row, n_col, n_chan))
X_test.shape

(1000, 100, 100, 2)

In [11]:
X_test[:,:,:,0]=X_test_1
X_test[:,:,:,1]=X_test_2
X_test.shape

(1000, 100, 100, 2)

# Shuffle X_train

In [12]:
arr = np.arange(y_train.shape[0])
np.random.shuffle(arr)
X_train = X_train[arr,:,:,:]
y_train = y_train[arr]

### 3. Rescale the Images by Dividing Every Pixel in Every Image by 255

In [13]:
# rescale [0,255] --> [0,1]
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

### 4.  Break Dataset into Training, Testing, and Validation Sets

In [14]:
# break training set into training and validation sets
(X_train, X_valid) = X_train[:1800], X_train[1800:]
(y_train, y_valid) = y_train[:1800], y_train[1800:]

# print shape of training set
print('x_train shape:', X_train.shape)

# print number of training, validation, and test images
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_valid.shape[0], 'validation samples')

x_train shape: (1800, 100, 100, 2)
1800 train samples
1000 test samples
400 validation samples


In [15]:
n_obs, n_row, n_col, n_chan = X_train.shape

In [16]:
print(n_obs, n_row, n_col, n_chan)

1800 100 100 2


### 5. Define the Model Architecture 

In [20]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# "valid" means no padding. "same" results in padding
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='valid', activation='relu', strides=(1,1), input_shape=(n_row, n_col, n_chan)))
model.add(MaxPooling2D(pool_size=2, strides=(1,1)))
model.add(Conv2D(filters=32, kernel_size=2, padding='valid', activation='relu', strides=(2,2)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='valid', activation='relu', strides=(1,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='valid', activation='relu', strides=(1,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 99, 99, 16)        144       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 98, 98, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 49, 49, 32)        2080      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 23, 23, 64)        8256      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 10, 10, 128)       32896     
__________

### 6. Compile the Model 

In [21]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

### 7. Train the Model 

In [22]:
model_path = 'model_9.weights.best.hdf5'

In [23]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, 
                               save_best_only=True)
hist = model.fit(X_train, y_train, batch_size=32, epochs=20,
          validation_data=(X_valid, y_valid), callbacks=[checkpointer], 
          verbose=2, shuffle=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1800 samples, validate on 400 samples
Epoch 1/20





 - 22s - loss: 0.7206 - acc: 0.4961 - val_loss: 0.6902 - val_acc: 0.5450

Epoch 00001: val_loss improved from inf to 0.69024, saving model to model_9.weights.best.hdf5
Epoch 2/20
 - 20s - loss: 0.6912 - acc: 0.5528 - val_loss: 0.6748 - val_acc: 0.6425

Epoch 00002: val_loss improved from 0.69024 to 0.67476, saving model to model_9.weights.best.hdf5
Epoch 3/20
 - 21s - loss: 0.6710 - acc: 0.6006 - val_loss: 0.6694 - val_acc: 0.6025

Epoch 00003: val_loss improved from 0.67476 to 0.66938, saving model to model_9.weights.best.hdf5
Epoch 4/20
 - 21s - loss: 0.6438 - acc: 0.6444 - val_loss: 0.7110 - val_acc: 0.5700

Epoch 00004: val_loss did not improve from 0.66938
Epoch 5/20
 - 20s - loss: 0.6335 - acc: 0.6389 - val_loss: 0.6758 - val_acc: 0.5975

Epoch 00005: val_loss did not improve from 0.66938
Epoch 6/20
 - 20s - loss: 0.6032 -

In [24]:
model_path

'model_9.weights.best.hdf5'

### 8. Load the Model with the Best Validation Accuracy

In [25]:
# load the weights that yielded the best validation accuracy
model.load_weights(model_path)

### 9. Calculate Classification Accuracy on Test Set

In [26]:
# evaluate and print test accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.706


In [27]:
model.predict(X_test)

array([[0.30931488, 0.6906851 ],
       [0.91027784, 0.08972216],
       [0.5426848 , 0.45731518],
       ...,
       [0.23233077, 0.7676692 ],
       [0.72518194, 0.27481803],
       [0.40180582, 0.5981942 ]], dtype=float32)

In [ ]:
model_path